In [ ]:
import pandas as pd
import numpy as np
import subprocess
import random
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor

In [109]:
def func(x):
    return x[1]*x[6] + x[8]/x[9]*np.sqrt(x[6]/x[7]) + (np.pi)*np.sqrt(x[2]) + 1/np.sin(x[3]) + np.log(x[2]+x[4]) 

# Define constants

In [110]:
INIT_COUNT = 2000
COUNT_TO_ADD = 50
TOT_SIZE = 1000000
PATH_TO_DATA = "all/public.data"
TARGET = np.zeros(TOT_SIZE)
list_of_initial_index = []
PFR = RandomForestRegressor(n_estimators = 150, n_jobs = -1)
BAGG = BaggingRegressor(n_estimators = 150, n_jobs = -1)
GDR = GradientBoostingRegressor(n_estimators =150, learning_rate = 0.075)
ADA = AdaBoostRegressor(n_estimators = 150, learning_rate = 0.01)
models = [PFR, BAGG, GDR, ADA]
sample = pd.read_csv("all/public.sample.csv")

# Read data

In [111]:
data = pd.read_csv(PATH_TO_DATA, 
                   sep= " ", 
                   names=["feature_{}".format(i) for i in range(0, 10)])

In [112]:
data.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9
0,8.401877,3.943829,7.830992,7.984400,9.116474,1.975514,3.352228,7.682296,2.777747,5.539700
1,4.773971,6.288709,3.647845,5.134009,9.522297,9.161951,6.357117,7.172969,1.416026,6.069689
2,0.163006,2.428868,1.372316,8.041768,1.566791,4.009444,1.297904,1.088088,9.989245,2.182569
3,5.129324,8.391122,6.126398,2.960316,6.375523,5.242872,4.935830,9.727750,2.925168,7.713577
4,5.267450,7.699138,4.002286,8.915295,2.833147,3.524583,8.077245,9.190265,0.697553,9.493271


# First init 2000 points to start active learning

In [113]:
list_of_initial_index = []

for _ in range(INIT_COUNT):
    now_ind = np.random.randint(TOT_SIZE)
    list_of_initial_index.append(now_ind)




for i in range(INIT_COUNT):
    temp_pont = np.array(data.iloc[list_of_initial_index[i]])
    value = func(temp_pont)
    TARGET[list_of_initial_index[i]] = value

In [114]:
print("Len of training data: {}".
      format(len(np.unique(list_of_initial_index))))


Len of training data: 1999


# Start fitting

In [115]:
inde = 0

new_final_ind = np.unique(list_of_initial_index)

In [116]:
def fill_target(new_ind, target):
    for ind in new_ind:
        temp_pont = np.array(data.iloc[ind])
        value = func(temp_pont)
        target[ind] = value
        

In [117]:
new_final_ind = list(new_final_ind)

In [121]:
for KDP in tqdm(range(1000)):
    inde+=1
    #Get the spread of predictions of the algorithm committee.
    
    #print("Len of training data: {}".
      #format(len(new_final_ind))
    
    X_train_new = data.iloc[new_final_ind]
    y_train_new = TARGET[new_final_ind]
    
    #print("Fitting algorithm")
    GDR.fit(X_train_new, y_train_new)
    pred = GDR.predict(X_train_new)
    
    #Looking for points with max-size errors
    
    err = np.abs(pred - y_train_new)
    #print("Fitting algorithm on errors")
   
    GDR.fit(X_train_new, err)
    pred_errs = np.abs(GDR.predict(data))
    pred_errs[new_final_ind] = 0 
    
    #print("Getting points with max errors")
    new_ind = pred_errs.argsort()[::-1][:COUNT_TO_ADD]
    
    
    
    
    
    
    
    
    
    #print("Getting new target values")
    
    fill_target(new_ind, TARGET)
    new_final_ind = list(np.unique(new_final_ind + list(new_ind)))
    
            
            
    if (KDP % 100 == 0):
        print("Size of TARGET on {} step is: {}".\
              format(KDP, len(TARGET[TARGET!=0])))
        
        print("Making submit")
        
        sample.Expected = TARGET

        sample.to_csv("sub_{}".format(KDP), index = False)
        
    

  0%|          | 1/1000 [00:02<36:04,  2.17s/it]

Size of TARGET on 0 step is: 2049


 10%|█         | 101/1000 [04:00<35:38,  2.38s/it]

Size of TARGET on 100 step is: 7049


 20%|██        | 201/1000 [09:58<39:38,  2.98s/it]

...

Size of TARGET on 300 step is: 17049


 40%|████      | 401/1000 [27:17<40:46,  4.08s/it]

Size of TARGET on 400 step is: 22049


 47%|████▋     | 472/1000 [35:57<40:13,  4.57s/it]

KeyboardInterrupt: 

# Final predict

In [122]:
sample = pd.read_csv("all/public.sample.csv")

In [123]:
sample.Expected = TARGET

sample.to_csv("sub_4", index = False)